In [1]:
BASE_DIR = '../'*3
RUN_DIR = BASE_DIR+"code/CNN/cnn_basic/cnn_basic_model.ipynb"
%run {RUN_DIR}

In [33]:
class CnnRegModel(CnnBasicModel):
    def __init__(self, name, dataset, hconfigs, show_maps=False,
                l2_decay=0, l1_decay=0):
        self.l2_decay = l2_decay
        self.l1_decay = l1_decay
        self.use_flatten = False
        super(CnnRegModel, self).__init__(name, dataset, hconfigs, show_maps)
        
    def exec_all(self, epoch_count=10, batch_size = 10, learning_rate = 0.001,
                 report=0, show_cnt = 3, show_params=False,num_workers=0):
        super(CnnRegModel, self).exec_all(epoch_count, batch_size, 
                                         learning_rate, report, show_cnt, num_workers)
        
        if show_params : self.show_param_dist()

In [23]:
class AddGaussianNoise(nn.Module):
    def __init__(self, mean=0., std=1.):
        super(AddGaussianNoise, self).__init__()
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

In [3]:
def cnn_reg_show_param_dist(self):
    params = self.collect_params().cpu().detach().numpy()
    
    mu = np.mean(params)
    sigma = np.sqrt(np.var(params))
    plt.hist(params, 100, density=True, facecolor='g', alpha=0.75)
    plt.axis([-0.2, 0.2, 0, 20.0])
    plt.text(0.08, 15.0, 'mu={:5.3f}'.format(mu))
    plt.text(0.08, 13.0, 'sigma={:5.3f}'.format(sigma))
    plt.grid(True)
    plt.show()
    
    total_count = len(params)
    near_zero_count = len(list(x for x in params if -1e-5 <= x <= 1e-5))
    print('Near 0 parameters = {:4.1f}%({}/{})'.
        format(near_zero_count/total_count*100, near_zero_count, total_count))

#1 . pm_output에 있는 w값
# 2.hidden에 있는 값 -> w, k(커널)

        
def cnn_reg_collect_params(self):
    param_list=[]

    for layer in self.model.layers:
        
        if isinstance(layer, nn.Linear)|isinstance(layer, nn.Conv2d):
            param_list += [layer.weight.flatten()]
            
#             == 'weight': param_list+=param.flatten().tolist()
    
#     print(len(param_list), type(param_list))
    param_list=torch.cat(param_list)
    return param_list

CnnRegModel.show_param_dist = cnn_reg_show_param_dist
CnnRegModel.collect_params = cnn_reg_collect_params

In [ ]:
def cnn_reg_forward_extra_cost(self):
#     extra = super(CnnRegModel, self).forward_extra_cost(y)
    if self.l1_decay>0:
        params = self.model.parameters()
        extra = sum(p.abs().sum()for p in params)
        
        return self.l1_decay * extra
    
    return self.l1_decay
CnnRegModel.forward_extra_cost = cnn_reg_forward_extra_cost

In [34]:
def cnn_reg_alloc_dropout_layer(self, input_shape, hconfig):
    
    keep_prob = get_conf_param(hconfig, 'keep_prob', 1.0)
    assert keep_prob > 0 and keep_prob <= 1
    if len(input_shape) == 3:
        pm = nn.Dropout2d(p=keep_prob)
    return [pm], input_shape


CnnRegModel.alloc_dropout_layer = cnn_reg_alloc_dropout_layer

In [ ]:
def cnn_reg_alloc_noise_layer(self, input_shape, hconfig):
    noise_type = get_conf_param(hconfig, 'type', 'normal')
    m = get_conf_param(hconfig, 'mean', 0.)
    s = get_conf_param(hconfig, 'std', 1.0)
    ratio = get_conf_param(hconfig, 'ratio', 1.0)

    assert noise_type == 'normal'
    pm = AddGaussianNoise(mean=m,std=s)
    
    return [pm], input_shape

CnnRegModel.alloc_noise_layer = cnn_reg_alloc_noise_layer

In [ ]:
def cnn_reg_alloc_batch_normal_layer(self, input_shape, hconfig):
    rescale=get_conf_param(hconfig, 'rescale',True)
    if len(input_shape) == 3 :
        chn,_,_ = input_shape
        ychn = get_conf_param(hconfig, 'chn',chn)
        bc_affine = get_conf_param(hconfig, 'affine', rescale)
        bc_epsilon = get_conf_param(hconfig, 'epsilon', 1e-5)
        bc_momentum = get_conf_param(hconfig, 'momentum', 0.1)
    else:
        assert len(input_shape)==2
        chn,_ = input_shape
        ychn = get_conf_param(hconfig, 'chn',chn)
        bc_affine = get_conf_param(hconfig, 'affine', rescale)
        bc_epsilon = get_conf_param(hconfig, 'epsilon', 1e-5)
        bc_momentum = get_conf_param(hconfig, 'momentum', 0.1)
    
#     bn_dim = input_shape[-1]
#     pm['mavg'] = np.zeros(bn_dim)
#     pm['mvar'] = np.ones(bn_dim)
#     if rescale:
#         pm['scale'] = np.ones(bn_dim)
#         pm['shift'] = np.zeros(bn_dim)
        
    if len(input_shape) == 3 :
        pm = nn.BatchNorm2d(ychn,affine=bc_affine,momentum=bc_momentum,eps=bc_epsilon)
    if len(input_shape) == 2:
        pm =  nn.BatchNorm1d(ychn,affine=bc_affine,momentum=bc_momentum,eps=bc_epsilon)
    return [pm], input_shape



CnnRegModel.alloc_batch_normal_layer = cnn_reg_alloc_batch_normal_layer


In [ ]:
# nn.BatchNorm2d()`

In [ ]:
# nn.BatchNorm2d(ychn, affine=bc_affine,momentum=bc_momentum,eps=bc_epsilon)

In [3]:
def get_optim(self, use_adam):
    
    if use_adam:
        optim=torch.optim.Adam(self.model.parameters(),lr=self.learning_rate,\
                         betas=(0.9,0.999),eps=1e-08, weight_decay=self.l2_decay)
    
    else :
        optim=torch.optim.SGD(self.model.parameters(),lr = self.learning_rate,weight_decay=self.l2_decay)
        
    
    return optim

CnnRegModel.get_optim = get_optim

In [ ]:
def weights_init(self,m):
    if isinstance(m, nn.Linear)|isinstance(m, nn.Conv2d):
        torch.nn.init.normal_(m.weight, mean=0., std=0.030)
        torch.nn.init.zeros_(m.bias)
    
CnnRegModel.weights_init = weights_init